In [1]:
import griddata as gd

net, const_load_household, time_steps, df_household = gd.setup_grid_powertech25(season='winter')

In [3]:
net.load

AttributeError: 'tuple' object has no attribute 'load'

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add all buses as nodes
for bus in net.bus.index:
    G.add_node(bus)

# Add edges based on line connections
for line in net.line.itertuples():
    from_bus = line.from_bus
    to_bus = line.to_bus
    G.add_edge(from_bus, to_bus)

# Add edges for transformer connections
for trafo in net.trafo.itertuples():
    hv_bus = trafo.hv_bus
    lv_bus = trafo.lv_bus
    G.add_edge(hv_bus, lv_bus)

# Find the root bus (e.g., the slack bus or transformer bus)
root_bus = net.ext_grid.bus.iloc[0]

# Debugging: Ensure the root bus exists in the graph
if root_bus not in G.nodes:
    print(f"Warning: Root bus {root_bus} is not in the graph.")
    print(f"Available nodes: {list(G.nodes)}")
else:
    print(f"Root bus {root_bus} found.")

# Generate a topological sort to ensure hierarchy
levels = {node: 0 for node in G.nodes}
visited = set()

def assign_levels(node, level):
    if node in visited:
        return
    visited.add(node)
    levels[node] = level
    for neighbor in G.neighbors(node):
        assign_levels(neighbor, level + 1)

# Assign levels starting from the root bus
assign_levels(root_bus, 0)

# Define a position based on levels
pos = {}
nodes_by_level = {}
for node, level in levels.items():
    if level not in nodes_by_level:
        nodes_by_level[level] = []
    nodes_by_level[level].append(node)

for level, nodes in nodes_by_level.items():
    y = -level  # Downward hierarchy
    x_positions = range(len(nodes))
    for x, node in zip(x_positions, nodes):
        pos[node] = (x, y)

# Plot the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=500,
    node_color="lightblue",
    font_size=10,
    arrowsize=10,
)

# Highlight the root bus
nx.draw_networkx_nodes(G, pos, nodelist=[root_bus], node_color="red", node_size=700)
plt.title("Hierarchical Representation of the Grid with Transformer Connections")
plt.show()


In [5]:
import pandapower as pp
# Buses and lines to remove
buses_to_drop = [14, 34, 9, 19]

# Remove lines associated with the buses to drop
lines_to_drop = net.line[
    (net.line.from_bus.isin(buses_to_drop)) | (net.line.to_bus.isin(buses_to_drop))
].index

# Drop the identified lines
pp.drop_lines(net, lines_to_drop)

# Drop the identified buses
pp.drop_buses(net, buses_to_drop)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add all buses as nodes
for bus in net.bus.index:
    G.add_node(bus)

# Add edges based on line connections
for line in net.line.itertuples():
    from_bus = line.from_bus
    to_bus = line.to_bus
    G.add_edge(from_bus, to_bus)

# Add edges for transformer connections
for trafo in net.trafo.itertuples():
    hv_bus = trafo.hv_bus
    lv_bus = trafo.lv_bus
    G.add_edge(hv_bus, lv_bus)

# Find the root bus (e.g., the slack bus or transformer bus)
root_bus = net.ext_grid.bus.iloc[0]

# Debugging: Ensure the root bus exists in the graph
if root_bus not in G.nodes:
    print(f"Warning: Root bus {root_bus} is not in the graph.")
    print(f"Available nodes: {list(G.nodes)}")
else:
    print(f"Root bus {root_bus} found.")

# Generate a topological sort to ensure hierarchy
levels = {node: 0 for node in G.nodes}
visited = set()

def assign_levels(node, level):
    if node in visited:
        return
    visited.add(node)
    levels[node] = level
    for neighbor in G.neighbors(node):
        assign_levels(neighbor, level + 1)

# Assign levels starting from the root bus
assign_levels(root_bus, 0)

# Define a position based on levels
pos = {}
nodes_by_level = {}
for node, level in levels.items():
    if level not in nodes_by_level:
        nodes_by_level[level] = []
    nodes_by_level[level].append(node)

for level, nodes in nodes_by_level.items():
    y = -level  # Downward hierarchy
    x_positions = range(len(nodes))
    for x, node in zip(x_positions, nodes):
        pos[node] = (x, y)

# Plot the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=500,
    node_color="lightblue",
    font_size=10,
    arrowsize=10,
)

# Highlight the root bus
nx.draw_networkx_nodes(G, pos, nodelist=[root_bus], node_color="red", node_size=700)
plt.title("Hierarchical Representation of the Grid with Transformer Connections")
plt.show()


In [94]:
# Set constant loads to 3 kW for all loads except R11 to R18
constant_loads = net.load.index.difference([1, 2, 3, 4, 5])  # Indices of loads R1, I2, C1, C12, C13, C14, C17, C18, C19, C20
net.load.loc[constant_loads, 'p_mw'] = 15/1000

In [95]:
# Initialize ConstControl controller to update values of the loads R11 to R18
profile_loads = net.load.index.intersection([0, 1, 2, 3, 4, 5])  # Indices of loads R11 to R18
const_load = control.ConstControl(net, element='load', element_index=profile_loads,
                                  variable='p_mw', data_source=ds, profile_name=["mult"]*len(profile_loads))


In [96]:
# Run a power flow for the initial condition to check if the network is properly configured
try:
    pp.runpp(net)
except pp.optimal_powerflow.OPFNotConverged:
    print("Initial power flow did not converge. Please check the network configuration.")

In [ ]:
# initialising the outputwriter to save data to excel files in the current folder. You can change this to .json, .csv, or .pickle as well
ow = timeseries.OutputWriter(net, output_path="./", output_file_type=".xlsx")
# adding vm_pu of all buses and line_loading in percent of all lines as outputs to be stored
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_load', 'q_mvar')


# starting the timeseries simulation for one day -> 96 15 min values.
timeseries.run_timeseries(net)
# now checkout the folders res_bus and res_line in your current working dir

In [98]:
# Load the results
vm_pu = pd.read_excel('res_bus/vm_pu.xlsx', index_col=0)
loading_percent = pd.read_excel('res_line/loading_percent.xlsx', index_col=0)
load_p_mw = pd.read_excel('res_load/p_mw.xlsx', index_col=0)
load_q_mvar = pd.read_excel('res_load/q_mvar.xlsx', index_col=0)


In [ ]:
import pandapower.plotting as plot
# Plot the grid
plot.simple_plot(net, show_plot=True)

In [ ]:
import matplotlib.pyplot as plt
# Generate the plots
plt.figure(figsize=(10, 5))
plt.plot(vm_pu, label=vm_pu.columns)
plt.xlabel('Time Step')
plt.ylabel('Voltage Magnitude (p.u.)')
plt.title('Voltage Magnitude over Time')
plt.legend()
plt.show()


In [ ]:

plt.figure(figsize=(10, 5))
plt.plot(loading_percent, label=loading_percent.columns)
plt.xlabel('Time Step')
plt.ylabel('Loading Percent')
plt.title('Line Loading over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(load_p_mw, label=load_p_mw.columns)
plt.xlabel('Time Step')
plt.ylabel('Active Power Load (MW)')
plt.title('Active Power Load over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(load_q_mvar, label=load_q_mvar.columns)
plt.xlabel('Time Step')
plt.ylabel('Reactive Power Load (MVAr)')
plt.title('Reactive Power Load over Time')
plt.legend()
plt.show()